In [ ]:
# import pandas
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.qda import QDA
import tensorflow as tf

import pandas 
import pandas as pd
from sklearn import preprocessing

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
tf.logging.set_verbosity(tf.logging.INFO)

#Chaikin Oscillator  
def Chaikin(df):  
    ad = (2 * df['ClosePrice'] - df['HighPrice'] - df['LowPrice']) / (df['HighPrice'] - df['LowPrice']) * df['TotalVolume']  
    Chaikin = pd.Series(pd.ewma(ad, span = 3, min_periods = 2) - pd.ewma(ad, span = 10, min_periods = 9), name = 'Chaikin')  
    df = df.join(Chaikin)  
    return df
## Bolinger Bands
def bbands(price, length=30, numsd=2):
    """ returns average, upper band, and lower band"""
    ave = pd.stats.moments.rolling_mean(price,length)
    sd = pd.stats.moments.rolling_std(price,length)
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

#Create output labels
def label(df, numberOfRowsAhead, priceDiffRange):
    # Create a list to store the data
    labelPosChang = []
    labelNegChang = []
    labelNoChang = []
    for i in range(0, len(df)):
       price=df.iloc[i]['ClosePrice']
       rowAheadCount=0
       if i == len(df)-1:
         print("End  of array")  
         labelPosChang.append(0)
         labelNegChang.append(0)
         labelNoChang.append(1)
         break 
          
       for j in range(i+1,i+numberOfRowsAhead):
        if j > len(df)-1:
           print("End  of array")  
           labelPosChang.append(0)
           labelNegChang.append(0)
           labelNoChang.append(1)
           break 
        priceDiff=df.iloc[j]['ClosePrice'] - price
         
        rowAheadCount = rowAheadCount +1 
        if priceDiff > priceDiffRange:
           labelPosChang.append(1)
           labelNegChang.append(0)
           labelNoChang.append(0)
           break
           
        elif priceDiff < -1*priceDiffRange:
           labelPosChang.append(0)
           labelNegChang.append(1)
           labelNoChang.append(0)
           break 
        elif numberOfRowsAhead-1 == rowAheadCount:
           labelPosChang.append(0)
           labelNegChang.append(0)
           labelNoChang.append(1)
           break
          
    print ('Done')
    print(len(labelPosChang))
    print(len(labelNegChang))
    print(len(df))
    df['Poschg'] = labelPosChang
    df['Negchg'] = labelNegChang
    df['Nochg'] = labelNoChang
   
    return df

## Generate CNN data using historical data 
def generateCNNInputData(df,previousrows):
       X=[]
       y=[]
       for i in range(previousrows-1, len(df)):
            ## to do check whether the right element is selected
            #print(df.iloc[i-previousrows+1:i+1][['OpenPrice','HighPrice','LowPrice','TotalVolume','priceChange','movingAverage','upperBB','lowerBB','movingAverage5','upperBB5','lowerBB5','Chaikin']].as_matrix().flatten())
            X.append(df.iloc[i-previousrows+1:i+1][['OpenPrice','HighPrice','LowPrice','TotalVolume','priceChange','movingAverage','upperBB','lowerBB','movingAverage5','upperBB5','lowerBB5','Chaikin']].as_matrix().flatten())
            y.append(df.iloc[i][['Poschg','Negchg','Nochg']].as_matrix().flatten())
            
            
       print("(X[0]")
       print(X[:0])
       #### Data Preprocessing using MinMaxScaler
       min_max_scaler = preprocessing.MinMaxScaler()
       X = min_max_scaler.fit_transform(X)
       return X,y
            




data = pandas.read_csv('C://Users/thame_000/DailyTickSandPNov112017.csv')
data=data[data['Ticker']=='MSFT']

data=Chaikin(data)
data['priceChange'] = data['ClosePrice']- data['ClosePrice'].shift() 
data['priceChangePred'] = data['ClosePrice'].shift(-1) - data['ClosePrice']

data=label(data,5,0.02)

 
          


data['movingAverage'], data['upperBB'], data['lowerBB'] = bbands(data.ClosePrice, length=30, numsd=1)
data['movingAverage5'], data['upperBB5'], data['lowerBB5'] = bbands(data.ClosePrice, length=5, numsd=1)

data.fillna(0, inplace=True)

print(data.sample)
            
X,y=generateCNNInputData(data,5)

input_size=len(X[0])
input_size=60

### Split into trai and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


dropout = 0.9 # Dropout, probability to keep units
def weight_variable(shape):
  #initial = tf.truncated_normal(shape, stddev=0.1)
  #initial =  tf.random_normal(shape)
  return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
  #initial = tf.constant(0.1, shape=shape)
  return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
                        
input_size=60
x=tf.placeholder('float',[None,input_size],name="x")
y_ = tf.placeholder(tf.float32, shape=[None, 3],name="y_")
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
dropout = 0.8 # Dropout, probability to keep units

## Create CNN Network
def neural_network_model(x,keep_prob,numberofcolumns ):
        numberofbackrecord=5
        numberoffilters1=32
        numberoffilters2=64
        print("numberofcolumns")
        print(numberofcolumns)
              
        W_conv1 = weight_variable([1, numberofbackrecord, 1, numberoffilters1])
        b_conv1 = bias_variable([numberoffilters1])
        x_image = tf.reshape(x, [-1,numberofcolumns,numberofbackrecord,1])
        
        
        ## conv 1 and max layer 1 
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        h_pool1=h_conv1
        
        W_conv2 = weight_variable([1, numberofbackrecord, numberoffilters1, numberoffilters2])
        b_conv2 = bias_variable([numberoffilters2])

        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        h_pool2=h_conv2
        
        ## dense layer1
        W_fc1 = weight_variable([numberofcolumns * numberofbackrecord * numberoffilters2, 1024])
        b_fc1 = bias_variable([1024])

        
        h_pool2_flat = tf.reshape(h_pool2, [-1, numberofcolumns * numberofbackrecord * numberoffilters2])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        
         ## drop layer
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        
        W_fc2 = weight_variable([1024, 3])
        b_fc2 = bias_variable([3])

        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
        
        
      
        return y_conv

def train_neural_network(x, keep_prob,input_size):
    prediction=neural_network_model(x, keep_prob,input_size)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction,y_))
    #learning rate defaulted
    optimizer=tf.train.AdamOptimizer().minimize(cost)
    logs_path="C:/Users/thame_000/tbcnn"
    writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())
    hm_epochs=200
    
    countMin = 0 
    minLossEpoch = 1000
    dropout=0.9
    correct=tf.equal(tf.argmax(prediction,1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct,'float'))
    maxAccuracy=0;
    
    with tf.Session() as sess:
       sess.run(tf.initialize_all_variables())
       tf.scalar_summary("trainaccuracy", accuracy)
       
       # merge all summaries into a single "operation" which we can execute in a session 
       summary_op = tf.merge_all_summaries()
        
       test_summary=tf.summary.scalar("testaccuracy", accuracy)
    
       for epoch in range(hm_epochs):
           epoch_loss=0
           batchSize=5
           batchStartIndex= 0
           batchEndIndex= batchSize
           batchCount=0
           
           lengthOfTrainingSet=len(X_train)
           #print("Length training set",lengthOfTrainingSet)
           while batchEndIndex <= lengthOfTrainingSet:
               batchCount=batchCount+1
               X_train_batch=X_train[batchStartIndex:batchEndIndex]
               y_train_batch=y_train[batchStartIndex:batchEndIndex]
               _,c,summary=sess.run([optimizer,cost, summary_op],feed_dict={x:X_train_batch,y_:y_train_batch, keep_prob: dropout})
               #print('Epoch',epoch,'completed out of',hm_epochs,'loss_epoch:',c)
               valid_acc, valid_summ = sess.run(
                            [accuracy, test_summary],
                                feed_dict={x:X_test,y_:y_test, keep_prob: dropout})
               writer.add_summary(valid_summ, epoch *  batchSize + batchCount)
               currAccuracy=valid_acc
               #print('Accuracy:',currAccuracy)
               if epoch == 0:
                  maxAccuracy=currAccuracy
               if currAccuracy >  maxAccuracy:
                  maxAccuracy=currAccuracy
                  print('maxAccuracy:',maxAccuracy)
               #if countMin > 100 :
                 #break  q
               if batchEndIndex >lengthOfTrainingSet-1 :
                 break
               batchStartIndex=batchEndIndex
               batchEndIndex=batchEndIndex+batchSize
               if batchEndIndex >lengthOfTrainingSet-1 :
                  batchEndIndex=lengthOfTrainingSet
                # write log
               writer.add_summary(summary, epoch *  batchSize + batchCount)
            
       print('maxAccuracy',maxAccuracy)
       
    


 

#12 is number of columns
numofinput=12

train_neural_network(x, keep_prob,numofinput)

C:\Users\thame_000\Anaconda3new\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\thame_000\Anaconda3new\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


(391, 10)
Index(['Date', 'Timestamp', 'Ticker', 'OpenPrice', 'HighPrice', 'LowPrice',
       'ClosePrice', 'TotalVolume', 'TotalQuantity', 'TotalTradeCount'],
      dtype='object')


C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(adjust=True,ignore_na=False,min_periods=2,span=3).mean()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(adjust=True,ignore_na=False,min_periods=9,span=10).mean()


End  of array
End  of array
End  of array
End  of array
Done
391
391
391
<bound method NDFrame.sample of            Date Timestamp Ticker  OpenPrice  HighPrice  LowPrice  ClosePrice  \
70256  20151117  09:31:00   MSFT      53.17      53.52     53.17       53.31   
70257  20151117  09:32:00   MSFT      53.32      53.44     53.32       53.38   
70258  20151117  09:33:00   MSFT      53.38      53.39     53.14       53.16   
70259  20151117  09:34:00   MSFT      53.15      53.25     53.11       53.17   
70260  20151117  09:35:00   MSFT      53.17      53.18     53.08       53.08   
70261  20151117  09:36:00   MSFT      53.08      53.15     53.05       53.11   
70262  20151117  09:37:00   MSFT      53.11      53.18     53.10       53.16   
70263  20151117  09:38:00   MSFT      53.16      53.17     53.12       53.17   
70264  20151117  09:39:00   MSFT      53.17      53.20     53.14       53.19   
70265  20151117  09:40:00   MSFT      53.19      53.24     53.18       53.21   
70266  20151117

C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:26: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=30,center=False).mean()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:27: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=30,center=False).std()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:26: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:27: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).std()


(X[0]
[]
numberofcolumns
12
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.merge.
maxAccuracy: 0.551282
maxAccuracy: 0.666667
maxAccuracy: 0.692308
